In [ ]:
import numpy as np
import dolfin as fe
import unray as ur

def mesh_from_fenics(obj):
    if isinstance(obj, fe.Function):
        obj = obj.function_space().mesh()
    if isinstance(obj, fe.MeshFunctionSizet):
        obj = obj.mesh()
    assert isinstance(obj, fe.Mesh)
    cells = obj.cells()
    points = obj.coordinates()
    cells = np.asarray(cells, dtype="int32")
    points = np.asarray(points, dtype="float32")
    mesh = ur.Mesh(cells=cells, points=points)
    return mesh

def field_from_fenics(obj, mesh):
    assert isinstance(obj, fe.Function)
    assert isinstance(mesh, ur.Mesh)
    # TODO: support P0 and D1 spaces
    values = obj.compute_vertex_values()
    values = np.asarray(values, dtype="float32")
    field = ur.Field(mesh=mesh, values=values, space="P1")
    return field

def indicator_field_from_fenics(obj, mesh):
    assert isinstance(obj, fe.MeshFunctionSizet)
    assert isinstance(mesh, ur.Mesh)
    values = obj.array()
    values = np.asarray(values, dtype="int32")
    space = "I%d" % obj.dim()
    field = ur.IndicatorField(mesh=mesh, values=values, space=space)
    return field

In [ ]:
mesh = fe.UnitCubeMesh(1, 1, 1)

V0 = fe.FunctionSpace(mesh, "DP", 0)
f0 = fe.Function(V0)
f0.interpolate(fe.Expression("x[0]", degree=1))

V1 = fe.FunctionSpace(mesh, "DP", 1)
f1 = fe.Function(V1)
f1.interpolate(fe.Expression("x[0]", degree=1))

V2 = fe.FunctionSpace(mesh, "P", 1)
f2 = fe.Function(V2)
f2.interpolate(fe.Expression("x[0]", degree=1))

cf = fe.CellFunctionSizet(mesh)
cf.set_all(1)

ff = fe.FacetFunctionSizet(mesh)
ff.set_all(1)

umesh = mesh_from_fenics(mesh)
g0 = field_from_fenics(f0, mesh=umesh)
g1 = field_from_fenics(f1, mesh=umesh)
g2 = field_from_fenics(f2, mesh=umesh)
ci = indicator_field_from_fenics(cf, mesh=umesh)
fi = indicator_field_from_fenics(ff, mesh=umesh)

assert g0.space == "P1"  # TODO: P0
assert g1.space == "P1"
assert g2.space == "P1"  # TODO: D0
assert ci.space == "I3"
assert fi.space == "I2"

In [ ]:
lut = ur.ArrayColorLUT(values=[[0,0,1], [1,0,0]])
color = ur.ColorField(field=g1, lut=lut)
restrict = ur.ScalarIndicators(field=ci, value=1)
plot = ur.SurfacePlot(mesh=umesh, restrict=restrict, color=color)
plot